<h1 style="text-align:center;"> DFS event prediction using NEAT </h1>

---

<h1 style="text-align:center;">Reading Data</h1>
<hr style="opacity: 0.3">

In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("./data/Merged Colorectal Cancer Patient Data.csv", sep=",")
df

,ID_REF,Age (in years),Dukes Stage,Gender,Location,DFS (in months),DFS event,Adj_Radio,Adj_Chem,117_at,...,1554911_at,1554912_at,1554914_at,1554915_a_at,1554916_a_at,1554918_a_at,1554919_s_at,1554920_at,213418_at,1554921_a_at
0,GSM877126,62.0,A,Male,Left,108.0,0.0,1.0,0.0,6.029834,...,3.504809,2.856109,3.130712,5.603126,3.091348,4.009692,4.350400,2.512026,7.494499,4.030978
1,GSM877127,77.0,B,Male,Left,40.0,1.0,1.0,0.0,6.344321,...,3.957833,2.988643,3.484177,6.601744,3.141737,4.679732,4.508760,2.427697,8.156995,2.827090
2,GSM877128,66.0,C,Female,Left,49.0,0.0,1.0,0.0,5.023127,...,3.749518,2.610840,3.406207,6.003205,3.469987,4.781843,4.435054,2.531300,6.058687,2.930454
3,GSM877129,72.0,D,Female,Left,45.0,0.0,1.0,1.0,4.612727,...,3.610429,2.728274,3.534871,5.858326,3.372291,3.653575,5.014994,2.608755,5.488072,2.949848
4,GSM877130,75.0,C,Male,Left,40.0,0.0,0.0,1.0,5.865944,...,3.520434,2.664744,3.314746,5.388115,3.322147,4.666360,5.013006,2.657130,7.410725,3.054697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,GSM877184,54.0,C,Male,Left,22.0,1.0,0.0,0.0,4.835558,...,3.637413,2.787781,3.519436,6.026848,3.404074,4.103411,4.303329,2.422700,6.072618,2.853109
58,GSM877185,48.0,D,Male,Right,64.0,0.0,1.0,0.0,5.694010,...,3.733335,3.473473,3.237316,6.167214,3.452096,4.724017,4.470426,2.634358,7.175365,2.667533
59,GSM877186,65.0,C,Male,Left,25.0,1.0,1.0,0.0,8.661629,...,3.436002,2.706722,3.349033,6.238026,3.109042,6.897634,3.797626,2.756086,10.121831,2.971542
60,GSM877187,71.0,A,Male,Rectum,58.0,1.0,1.0,0.0,5.702601,...,3.438260,2.539084,3.334590,5.842051,3.496104,4.883845,4.661893,2.581452,7.232090,2.719928


----

<h1 style="text-align:center"> Preprocessing the data </h1>

## Creating dummies

In [6]:
df_processed = pd.get_dummies(df, columns=['Gender',"Location","Dukes Stage"])
df_processed

,ID_REF,Age (in years),DFS (in months),DFS event,Adj_Radio,Adj_Chem,117_at,1007_s_at,1053_at,121_at,...,Gender_Female,Gender_Male,Location_Colon,Location_Left,Location_Rectum,Location_Right,Dukes Stage_A,Dukes Stage_B,Dukes Stage_C,Dukes Stage_D
0,GSM877126,62.0,108.0,0.0,1.0,0.0,6.029834,9.468893,7.187090,6.807582,...,0,1,0,1,0,0,1,0,0,0
1,GSM877127,77.0,40.0,1.0,1.0,0.0,6.344321,9.994204,7.705626,6.893130,...,0,1,0,1,0,0,0,1,0,0
2,GSM877128,66.0,49.0,0.0,1.0,0.0,5.023127,10.409823,7.934057,7.165602,...,1,0,0,1,0,0,0,0,1,0
3,GSM877129,72.0,45.0,0.0,1.0,1.0,4.612727,10.596921,7.446374,7.071417,...,1,0,0,1,0,0,0,0,0,1
4,GSM877130,75.0,40.0,0.0,0.0,1.0,5.865944,10.252547,7.141105,7.103383,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,GSM877184,54.0,22.0,1.0,0.0,0.0,4.835558,10.343581,7.471584,7.920227,...,0,1,0,1,0,0,0,0,1,0
58,GSM877185,48.0,64.0,0.0,1.0,0.0,5.694010,9.409749,7.488735,6.699796,...,0,1,0,0,0,1,0,0,0,1
59,GSM877186,65.0,25.0,1.0,1.0,0.0,8.661629,10.427298,7.512632,7.046389,...,0,1,0,1,0,0,0,0,1,0
60,GSM877187,71.0,58.0,1.0,1.0,0.0,5.702601,10.309094,7.476872,7.349701,...,0,1,0,0,1,0,1,0,0,0


# Changing the type of Age, DFS, ADJ's from float to int

In [7]:
labels = [
    "Age (in years)",
    "DFS (in months)",
    "DFS event",
    "Adj_Radio",
    "Adj_Chem",
]

for label in labels :
    df_processed[label] = df_processed[label].astype(int)
df_processed

,ID_REF,Age (in years),DFS (in months),DFS event,Adj_Radio,Adj_Chem,117_at,1007_s_at,1053_at,121_at,...,Gender_Female,Gender_Male,Location_Colon,Location_Left,Location_Rectum,Location_Right,Dukes Stage_A,Dukes Stage_B,Dukes Stage_C,Dukes Stage_D
0,GSM877126,62,108,0,1,0,6.029834,9.468893,7.187090,6.807582,...,0,1,0,1,0,0,1,0,0,0
1,GSM877127,77,40,1,1,0,6.344321,9.994204,7.705626,6.893130,...,0,1,0,1,0,0,0,1,0,0
2,GSM877128,66,49,0,1,0,5.023127,10.409823,7.934057,7.165602,...,1,0,0,1,0,0,0,0,1,0
3,GSM877129,72,45,0,1,1,4.612727,10.596921,7.446374,7.071417,...,1,0,0,1,0,0,0,0,0,1
4,GSM877130,75,40,0,0,1,5.865944,10.252547,7.141105,7.103383,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,GSM877184,54,22,1,0,0,4.835558,10.343581,7.471584,7.920227,...,0,1,0,1,0,0,0,0,1,0
58,GSM877185,48,64,0,1,0,5.694010,9.409749,7.488735,6.699796,...,0,1,0,0,0,1,0,0,0,1
59,GSM877186,65,25,1,1,0,8.661629,10.427298,7.512632,7.046389,...,0,1,0,1,0,0,0,0,1,0
60,GSM877187,71,58,1,1,0,5.702601,10.309094,7.476872,7.349701,...,0,1,0,0,1,0,1,0,0,0


In [14]:
Y = df_processed['DFS event']
Y

0     0
1     1
2     0
3     0
4     0
     ..
57    1
58    0
59    1
60    1
61    1
Name: DFS event, Length: 62, dtype: int32

In [15]:
drop_labels = [
   "DFS event",
   "ID_REF"
]
X = df_processed.drop(drop_labels, axis = 1)
X

,Age (in years),DFS (in months),Adj_Radio,Adj_Chem,117_at,1007_s_at,1053_at,121_at,1255_g_at,1294_at,...,Gender_Female,Gender_Male,Location_Colon,Location_Left,Location_Rectum,Location_Right,Dukes Stage_A,Dukes Stage_B,Dukes Stage_C,Dukes Stage_D
0,62,108,1,0,6.029834,9.468893,7.187090,6.807582,3.084591,7.397013,...,0,1,0,1,0,0,1,0,0,0
1,77,40,1,0,6.344321,9.994204,7.705626,6.893130,3.118704,7.163797,...,0,1,0,1,0,0,0,1,0,0
2,66,49,1,0,5.023127,10.409823,7.934057,7.165602,2.984632,7.375224,...,1,0,0,1,0,0,0,0,1,0
3,72,45,1,1,4.612727,10.596921,7.446374,7.071417,2.912851,7.596233,...,1,0,0,1,0,0,0,0,0,1
4,75,40,0,1,5.865944,10.252547,7.141105,7.103383,3.383726,7.278244,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,54,22,0,0,4.835558,10.343581,7.471584,7.920227,2.888331,7.628249,...,0,1,0,1,0,0,0,0,1,0
58,48,64,1,0,5.694010,9.409749,7.488735,6.699796,3.201511,7.192814,...,0,1,0,0,0,1,0,0,0,1
59,65,25,1,0,8.661629,10.427298,7.512632,7.046389,2.650723,5.992824,...,0,1,0,1,0,0,0,0,1,0
60,71,58,1,0,5.702601,10.309094,7.476872,7.349701,2.858642,6.878941,...,0,1,0,0,1,0,1,0,0,0


----

<h1 style="text-align:center"> Creating a NEAT model to predict DFS event </h1>

In [40]:
# Neural Network modules
import neat
import sys 
import os
import pickle

# --------------------
import logging
logging.basicConfig(level=logging.DEBUG, filename="main_log.log")
# --------------------

GENERATION = 0
MAX_ACCURACY = 0
# Data are X, Y

In [46]:
def save_gene(gene, name):
    with open(f"./saves/{name}.pkl", "wb") as f:
        pickle.dump(gene, f)

In [53]:
def feed(genomes, config):
    global GENERATION, MAX_ACCURACY
    GENERATION += 1

    nets  = [] 
    genes = [] 
    
    # Initialize the neural network models for current generation
    for _, gene in genomes:
        net = neat.nn.FeedForwardNetwork.create(gene, config)
        gene.fitness = 0
        nets.append(net)
        genes.append(gene)
    
    # Iterating over genes
    for i, gene in enumerate(genes):
        # Iterating over our data namely rows
        rows = X.itertuples()
        correctly_guessed = 0
        for row in rows:
            idx = row[0]
            data = row[1:]

            output = nets[i].activate(data)
            choice = output.index(max(output))
            
            actual = Y[idx]
            if choice == actual:
                gene.fitness += 10
                correctly_guessed += 1 
            #----<log>
            logging.info(f"{idx} -> {choice}, {choice==actual}")
            # if choice != 0 or choice != 1:
            #     logging.error(f"Choice isn't 0 or 1 {choice=}")
            #----</log>
        g
        accuracy = (correctly_guessed/len(Y))*100
        MAX_ACCURACY = max(MAX_ACCURACY, accuracy)
    #----<log>
    logging.info(f"{GENERATION= }, {MAX_ACCURACY= }")
    #----</log>

    #----<save>
    max_this_generation = sorted(genes, key=lambda x: x.fitness, reverse=True)[0]
    save_gene(max_this_generation, f"Gene_{GENERATION}")
    #----</save>


In [55]:

# Load Config
# local_dir = os.path.dirname(__file__)
# config_path = os.path.join(local_dir,"./config.txt")
config_path = ("./config.txt")

config = neat.config.Config(neat.DefaultGenome,
                            neat.DefaultReproduction,
                            neat.DefaultSpeciesSet,
                            neat.DefaultStagnation,
                            config_path)

# Create Population And Add Reporters
population = neat.Population(config)
population.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
population.add_reporter(stats)
    

In [58]:
# Run Simulation For A Maximum of 1000 Generations
winner = population.run(feed, 1000)
save_gene(winner, "Best")


 ****** Running generation 0 ****** 

Population's average fitness: 279.70000 stdev: 56.01705
Best fitness: 460.00000 - size: (2, 3898) - species 1 - id 172
Average adjusted fitness: 0.356
Mean genetic distance 1.122, standard deviation 0.270
Population of 200 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   200    460.0    0.356     0
Total extinctions: 0
Generation time: 113.463 sec

 ****** Running generation 1 ****** 

Population's average fitness: 329.45000 stdev: 66.60479
Best fitness: 460.00000 - size: (2, 3898) - species 1 - id 172
Average adjusted fitness: 0.565
Mean genetic distance 1.454, standard deviation 0.434
Population of 200 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   168    460.0    0.565     1
     2    0    32       --       --     0
Total extinctions: 0
Generation time: 112.650 sec (113.057 average)

 ****** Running gen